# Подключение библиотек

In [ ]:
import httplib2
from bs4 import BeautifulSoup,SoupStrainer

import urllib.request
from lxml.html import fromstring
import numpy as np
import pandas as pd
import os
import csv
import requests
import shutil
import re

# Сбор данных

In [ ]:
name_movie = []
director_list = []
main_role_list = []
date_list = []
duration_list = []
url = 'https://en.wikipedia.org'
main_page = '/wiki/Academy_Award_for_Best_Picture'
responce=urllib.request.urlopen(url+main_page).read()
page=fromstring(responce)
for n in range(2, 13):
  for j in range(3,100):
    try:
      name = page.xpath(str('/html/body/div[3]/div[3]/div[5]/div[1]/table[')+str(n)+str(']//tr[')+str(j)+str(']/td[1]//text()'))[0]
      winner = page.xpath(str('/html/body/div[3]/div[3]/div[5]/div[1]/table[')+str(n)+str(']//tr[')+str(j)+str(']/@style'))[0].split('#')[1]
      if winner == "FAEB86":
        name_movie.append(name)
        href = page.xpath(str('/html/body/div[3]/div[3]/div[5]/div[1]/table[')+str(n)+str(']//tr[')+str(j)+str(']/td[1]//@href'))[0]
        film_url = url + href
        responce_film = urllib.request.urlopen(film_url).read()
        page_film = fromstring(responce_film)
        need_column = ['Directed by', 'Starring', 'Release dates','Release date', 'Running time']
        for i in range(3,30):
          try:
            information_about_film = page_film.xpath(str('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody/tr[')+str(i)+str(']//text()'))
            for i in need_column:
              if i == 'Directed by' and i == information_about_film[0]:
                director_list.append(information_about_film[1])
                need_column.remove(i)
              elif i == 'Starring' and i == information_about_film[0]:
                main_role_list.append(str(information_about_film[1:])[1:-1].replace("'", ""))
                need_column.remove(i) 
              elif i == 'Release dates' and i == information_about_film[0] or  i == 'Release date' and i == information_about_film[0]:
                if information_about_film[1] != '\n':
                  date_list.append(information_about_film[1])
                else:
                  date_list.append(information_about_film[2])
                need_column.remove(i)
              elif i == 'Running time' and i == information_about_film[0]:
                if information_about_film[1] =='Original release:':
                  duration_list.append(information_about_film[2])
                elif information_about_film[1] =='\n':
                  duration_list.append(information_about_film[2])
                else:
                  duration_list.append(information_about_film[1])
                need_column.remove(i)
          except:
            continue
        for g in need_column:
          if g == 'Running time':
            duration_list.append("None")
 
    except IndexError:
      continue    

# Формирование таблицы

In [ ]:
table = pd.DataFrame({'Название фильма': name_movie, "Режиссёр": director_list, "Исполнители главных ролей": main_role_list, "Дата выхода фильма": date_list, "Продолжительность": duration_list})

In [ ]:
table

,Название фильма,Режиссёр,Исполнители главных ролей,Дата выхода фильма,Продолжительность
0,Wings,William A. Wellman,"Clara Bow, Charles (Buddy) Rogers, Richard Arl...","August 12, 1927",111 minutes
1,The Broadway Melody,Harry Beaumont,"Charles King, Anita Page, Bessie Love","February 1, 1929",100 minutes
2,All Quiet on the Western Front,Lewis Milestone,"Lew Ayres, Louis Wolheim","April 21, 1930",152 minutes
3,Cimarron,Wesley Ruggles,"Richard Dix, Irene Dunne","January 26, 1931",124 minutes
4,Grand Hotel,Edmund Goulding,"Greta Garbo, John Barrymore, Joan Crawford, Wa...","April 12, 1932",112 minutes
...,...,...,...,...,...
88,Moonlight,Barry Jenkins,"\n, Trevante Rhodes, \n, André Holland, \n, Ja...","September 2, 2016",111 minutes
89,The Shape of Water,Guillermo del Toro,"\n, Sally Hawkins, \n, Michael Shannon, \n, Ri...","August 31, 2017",123 minutes
90,Green Book,Peter Farrelly,"\n, Viggo Mortensen, \n, Mahershala Ali, \n, L...","September 11, 2018",130 minutes
91,Parasite,Bong Joon-ho,"\n, Song Kang-ho, \n, Lee Sun-kyun, \n, Cho Ye...",21 May 2019,132 minutes
